# Bivariate interpolation

Perform a bivariate interpolation of gridded data points.

For this test, we will use the 2D field that defines the average sea surface.

---
**Warning**

The file `../tests/dataset/mss.nc` is an old version of the subsampled version of the
Mean Sea Surface Height distributed by CNES/CLS. Do not use it for scientific purposes,
download the latest updated high-resolution version instead [here](https://www.aviso.altimetry.fr/en/data/products/auxiliary-products/mss.html).

---

In [ ]:
import netCDF4

ds = netCDF4.Dataset("../tests/dataset/mss.nc")
lon = ds.variables["lon"]
lat = ds.variables["lat"]
var = ds.variables["mss"]

Now that the data is loaded into memory, we will build the objects that will manipulate the axes of this regular grid. These axes will be used to locate the different pixels on the regular grid as shown in the figure below.

<img src="map.svg" alt="Drawing" align="middle" style="width: 100%;"/>

We start the construction of the X [axis](https://pangeo-pyinterp.readthedocs.io/en/latest/api/pyinterp.core.html#pyinterp.core.Axis), representing in our case the longitudes of the grid. When instantiating the object, this axis is specified as a circle.

In [ ]:
import pyinterp

x_axis = pyinterp.Axis(lon[:], is_circle=True)
x_axis

Finally, we instantiate the Y axis, representing in our case the latitudes.

In [ ]:
y_axis = pyinterp.Axis(lat[:])
y_axis

In order to build, the object handling this grid, we will alter the matrix to be shaped as `mss(x-axis, y-axis)` and to set the undefined values to `NaN`.

In [ ]:
mss = var[:].T
mss[mss.mask] = float("nan")

Finally, we can build the final object handling our [grid](https://pangeo-pyinterp.readthedocs.io/en/latest/api/pyinterp.grid.html#pyinterp.grid.Grid2D).

In [ ]:
grid = pyinterp.Grid2D(x_axis, y_axis, mss.data)
grid

We will build a new grid that will be used to build a new interpolated grid.

In [ ]:
import numpy as np

# The coordinates used for interpolation are shifted to avoid using the
# points of the bivariate function.
mx, my = np.meshgrid(np.arange(-180, 180, 1) + 1 / 3.0,
                     np.arange(-89, 89, 1) + 1 / 3.0,
                     indexing='ij')

The original grid is [interpolated](https://pangeo-pyinterp.readthedocs.io/en/latest/api/pyinterp.bivariate.html#pyinterp.bivariate.bivariate) to the target grid using the default setting (bilinear interpolation).

Values can be interpolated with several methods: `bilinear`, `nearest`, and `inverse distance weighting`. Distance calculations, if necessary, are calculated using the [Haversine formula](https://en.wikipedia.org/wiki/Haversine_formula).

In [ ]:
interpolated_mss = pyinterp.bivariate(
    grid, mx.flatten(), my.flatten()).reshape(mx.shape)

Let's visualize the original grid and the result of the interpolation.

In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
%matplotlib inline

fig = plt.figure(figsize=(18, 9))
ax = fig.add_subplot(121, projection=ccrs.PlateCarree(central_longitude=180))
lons, lats = np.meshgrid(grid.x, grid.y, indexing='ij')
ax.pcolormesh(lons, lats, grid.array, cmap='jet',
              transform=ccrs.PlateCarree())
ax.coastlines()
ax.set_title("Original MSS")

ax = fig.add_subplot(122, projection=ccrs.PlateCarree())
ax.pcolormesh(mx, my, interpolated_mss, cmap='jet',
              transform=ccrs.PlateCarree())
ax.coastlines()
ax.set_title("Interpolated MSS")

## xarray support


An experimental module of the library simplifies the use of the library by using [xarray](http://xarray.pydata.org/en/stable/) and [CF](http://cfconventions.org/) information contained in dataset. Indeed, this function will automatically determine which axis represents the longitude, latitude and ensure that the processed matrix has the right shape. This allows the interpolator to be created much faster.

---

**Note**

An exception will be thrown, if the constructor is not able to determine which axes are the longitudes and latitudes.

---

In [ ]:
import pyinterp.backends.xarray
import xarray as xr

ds = xr.load_dataset("../tests/dataset/mss.nc")
interpolator = pyinterp.backends.xarray.Grid2D(ds.data_vars["mss"])
interpolator

This class allows access to all 2D interpolators using the [bivariate method](https://pangeo-pyinterp.readthedocs.io/en/latest/api/pyinterp.backends.xarray.html#pyinterp.backends.xarray.Grid2D.bivariate).

In [ ]:
mss = interpolator.bivariate(dict(lon=mx.flatten(), lat=my.flatten()))